In [1]:
%matplotlib inline
import numpy as np
import pylab as pl
import scipy.special as special
from scipy.integrate import quad

In [12]:
#want to plot the integrand in Eq. 60 of the paper
#with filled in values of I(...), delta(yn), 

#constants in natural units
m = 1.
c = 1.
theta_e = 10.
#nu_c = 1
B = 10.
e = 1
nu_c = e * B / (2. * np.pi * m * c)


#constants in Gaussian cgs units (values from harmony)
#m = 9.1e-28
#c = 3e10
#theta_e = 5.8e10
#nu_c = 8.4e7
#nu = 5e7


#variables to be set (frequency, observer angle, width of delta function, electron number density, 
# sum index n)

nu = 1
theta = np.pi/3.
epsilon = 0.01
n_e = 1

cos_xi = np.arange(-1., 1., 2./100)
gamma = np.arange(1, 11, 0.1)


def delta_function(n, gamma, cos_xi):
    beta = np.sqrt(1. - 1./gamma**2.)
    argument = (n * nu_c)/gamma - nu * (1 - beta * cos_xi * np.cos(theta))
    f = (1/(epsilon * np.sqrt(np.pi))) * np.exp(-argument**2./epsilon**2.)
    return f

def K_s(n, gamma, cos_xi):
    beta = np.sqrt(1. - 1./gamma**2.)
    M = (np.cos(theta) - beta * cos_xi)/np.sin(theta)
    N = beta * np.sqrt(1. - cos_xi**2)
    z = (nu * gamma * beta * np.sin(theta) * np.sqrt(1. - cos_xi**2))/nu_c
    K_xx = M**2 * (special.jv(n, z))**2
    K_yy = N**2 * (special.jvp(n, z, 1))**2
    f = K_xx + K_yy
    return f

#def multiply_factor(n, gamma, cos_xi):
#    f = delta_function(n, gamma, cos_xi) * K_s(n, gamma, cos_xi)
#    return f

def n_sum(gamma, cos_xi):
    n_max     = 2000
    total     = 0
    for i in range(1,n_max):
        total = total + delta_function(i, gamma, cos_xi) * K_s(i, gamma, cos_xi)
    return total

#def integral_n_over_30(gamma, cos_xi):
#    total,error = nquad(multiply_factor, [31., np.inf])
#    return total

def MJ_f(gamma):
    beta = np.sqrt(1. - 1./gamma**2.)
    prefactor_MJ  = 1. / (m**3 * c**3 * gamma**2 * beta)
    thermal_dist_prefactor = n_e / (4. * np.pi * theta_e)
    thermal_dist_exp = np.exp(- gamma/theta_e)
    thermal_dist_body = gamma * np.sqrt(gamma**2 - 1.)/special.kn(2., 1./theta_e)
    thermal_dist = thermal_dist_prefactor * thermal_dist_body * thermal_dist_exp
    MJ = prefactor_MJ * thermal_dist
    return MJ


def jnu_integrand(cos_xi, gamma):
    beta = np.sqrt(1. - 1./gamma**2.)
    
    prefactor_Jnu = 2. * np.pi * e**2 * nu**2 / c
    
    d_3_p = m**3 * c**3 * gamma**2 * beta * 2 * np.pi
    
    #I = prefactor_Jnu * MJ_f(gamma) * d_3_p * K_s(n, gamma, cos_xi)
    
    #f = delta_function(n, gamma, cos_xi) * I * (-1.) * np.sin(cos_xi)
    
    #to do integral, we need to change dcos(xi) into -sin(xi)dxi
    f = prefactor_Jnu * MJ_f(gamma) * n_sum(gamma, cos_xi) * d_3_p
    #f = prefactor_Jnu * MJ_f(gamma) * (n_sum(gamma, xi)) * d_3_p * (-1.) * np.sin(xi)
    return f

print(jnu_integrand(0.5, 12))

7.02531545641e-11
